In [3]:
import logging
import os.path
import argparse
from typing import Any, Union

# External modules
import xarray as xr
import numpy as np
from tqdm import tqdm

# Internal modules

def get_regridder(
        grid_in: xr.Dataset, out_res: float, method='bilinear'
) -> Any:
    import xesmf as xe
    grid_out = xr.Dataset(
        {
            'lat': (['lat'], np.arange(-90+out_res/2, 90, out_res)),
            'lon': (['lon'], np.arange(0, 360, out_res)),
        }
    )
    regridder = xe.Regridder(grid_in, grid_out, method=method,
                             periodic=True, reuse_weights=False)
    return regridder


def _regrid_loop(
        regridder: Any,
        ds: xr.Dataset,
        chunk_size: Union[int, None] = None
) -> xr.Dataset:
    if chunk_size is None:
        chunk_size = len(ds['time'])
    ds_regridded = []
    regrid_pbar = tqdm(range(0, len(ds['time']), chunk_size))
    for chunk in regrid_pbar:
        tmp_regridded = regridder(
            ds.isel(time=slice(chunk, chunk+chunk_size))
        )
        ds_regridded.append(tmp_regridded)
    ds_regridded = xr.concat(ds_regridded, dim='time')
    for var in ds_regridded:
        ds_regridded[var].attrs = ds[var].attrs
    ds_regridded.attrs = ds.attrs
    return ds_regridded


def regrid_ds(
        ds_ens: Union[xr.Dataset, None],
        chunk_size: Union[int, None] = None
) -> Union[xr.Dataset, None]:
    if ds_ens is not None:
        grid_in = ds_ens[['lat', 'lon']]
        regridder = get_regridder(grid_in, out_res=5.625)
        ds_regridded = _regrid_loop(regridder, ds_ens, chunk_size)
    else:
        ds_regridded = None
    return ds_regridded

In [2]:
import xesmf

ImportError: Numba could not be imported.
--------------------------------------------------------------------------------
If you are seeing this message and are undertaking Numba development work, you may need to rebuild Numba.
Please see the development set up guide:

https://numba.readthedocs.io/en/stable/developer/contributing.html.

--------------------------------------------------------------------------------
If you are not working on Numba development, the original error was: 'cannot import name '_typeconv' from 'numba.core.typeconv' (c:\Users\haris\miniconda3\envs\ens\lib\site-packages\numba\core\typeconv\__init__.py)'.
For help, please visit:

https://numba.readthedocs.io/en/stable/user/faq.html#numba-could-not-be-imported
